In [1]:
import os
import time
from functools import reduce
from ultralytics import YOLO
import numpy as np
import cv2
import pandas as pd
from torch import cuda

In [2]:
model_path = "../models/yolov8m-seg.pt"

In [3]:
# ALL_NAMES:
YOLO(model_path).names

{0: 'person',
 1: 'bicycle',
 2: 'car',
 3: 'motorcycle',
 4: 'airplane',
 5: 'bus',
 6: 'train',
 7: 'truck',
 8: 'boat',
 9: 'traffic light',
 10: 'fire hydrant',
 11: 'stop sign',
 12: 'parking meter',
 13: 'bench',
 14: 'bird',
 15: 'cat',
 16: 'dog',
 17: 'horse',
 18: 'sheep',
 19: 'cow',
 20: 'elephant',
 21: 'bear',
 22: 'zebra',
 23: 'giraffe',
 24: 'backpack',
 25: 'umbrella',
 26: 'handbag',
 27: 'tie',
 28: 'suitcase',
 29: 'frisbee',
 30: 'skis',
 31: 'snowboard',
 32: 'sports ball',
 33: 'kite',
 34: 'baseball bat',
 35: 'baseball glove',
 36: 'skateboard',
 37: 'surfboard',
 38: 'tennis racket',
 39: 'bottle',
 40: 'wine glass',
 41: 'cup',
 42: 'fork',
 43: 'knife',
 44: 'spoon',
 45: 'bowl',
 46: 'banana',
 47: 'apple',
 48: 'sandwich',
 49: 'orange',
 50: 'broccoli',
 51: 'carrot',
 52: 'hot dog',
 53: 'pizza',
 54: 'donut',
 55: 'cake',
 56: 'chair',
 57: 'couch',
 58: 'potted plant',
 59: 'bed',
 60: 'dining table',
 61: 'toilet',
 62: 'tv',
 63: 'laptop',
 64: 'mou

In [4]:
CLASSES_IDS = [
    2, # Car
    7, # Truck
    5, # Bus
]

In [5]:
device = "gpu" if cuda.is_available() else "cpu"


In [6]:

class YOLOSegmentation:
    def __init__(self, model_path, device):
        self.model = YOLO(model_path,)
        self.device = device

    def detect(self, img):
        height, width, channels = img.shape

        results = self.model.predict(
            source=img.copy(), save=False, save_txt=False, device=self.device,
            conf=0.3  # Default confidence is 0.25. We need more so that we will skip various trimmed machines that we don't expect to get
        )
        result = results[0]
        segmentation_contours_idx = []
        for seg in result.masks.xyn:
            seg[:, 0] *= width
            seg[:, 1] *= height
            segment = np.array(seg, dtype=np.int32)
            segmentation_contours_idx.append(segment)
        bboxes = np.array(result.boxes.xyxy.to(self.device), dtype="int")
        class_ids = np.array(result.boxes.cls.to(self.device), dtype="int")
        scores = np.array(result.boxes.conf.to(self.device), dtype="float").round(2)

        # bboxes = np.array(result.boxes.xyxy.cpu(), dtype="int")
        # class_ids = np.array(result.boxes.cls.cpu(), dtype="int")
        # scores = np.array(result.boxes.conf.cpu(), dtype="float").round(2)

        return bboxes, class_ids, segmentation_contours_idx, scores


In [7]:
def get_max_area(indexed_box1, indexed_box2):
    """
    Comparing function that will be passed to 'reduce'.
    We calculate area of two boxes and return biggest
    :param indexed_box1:
    :param indexed_box2:
    :return: one of boxes - tuple like `index, bbox`
    """
    b1_i, (b1_x, b1_y, b1_x2, b1_y2) = indexed_box1
    b2_i, (b2_x, b2_y, b2_x2, b2_y2) = indexed_box2
    b1_area = (b1_x2 - b1_x) * (b1_y2 - b1_y)
    b2_area = (b2_x2 - b2_x) * (b2_y2 - b2_y)

    # print(f"B1 AREA: {b1_area}")
    result = indexed_box1 if b1_area > b2_area else indexed_box2
    return result

In [8]:
def check_object(bboxes, classes):
    proper_boxes = []
    target_box = None, None
    for object_class, bbox in zip(classes, bboxes):
        # Calculate number of objects that may be cars
        if object_class in CLASSES_IDS:
            target_box = list(classes).index(object_class), bbox
            proper_boxes.append(bbox)

    # Now, if we have for than one car, we will continue checking
    if len(proper_boxes) > 1:
        # We will pick up the object with the biggest area occupied
        target_box = reduce(get_max_area, enumerate(bboxes))

    return target_box


In [13]:
def extract_contours(folder: str, output_dir: str, Segmentator: YOLOSegmentation, transform_to_drawings=False, skip_exist=False) -> None:
    logs = []
    for image_name in os.listdir(folder):
        if not (image_name.endswith(".jpg") or image_name.endswith(".jpeg")):
            continue
        start_path = folder + '/' + image_name
        end_path = f"{output_dir}/"
        os.makedirs(end_path, exist_ok=True)
        if skip_exist:
            if image_name in os.listdir(end_path):
                continue

        img = cv2.imread(start_path)
        if img is None:
            message = f"Can't read an image: {image_name}"
            print(Exception(message))
            logs.append(message)
            continue
        img = cv2.resize(img, None, fx=0.5, fy=0.5)

        bboxes, classes, segmentations, scores = Segmentator.detect(img)

        b_id, bbox = check_object(bboxes=bboxes, classes=classes)
        if bbox is None:
            message = f"Can't extract car at image: {image_name}"
            print(Exception(message))
            logs.append(message)
            continue
        points = np.array(segmentations[b_id])

        mask = np.zeros(img.shape[:2], dtype=np.uint8)

        cv2.drawContours(mask, [points], -1, (255, 255, 255), -1, cv2.LINE_AA)

        (x, y, x2, y2) = bbox


        img = cv2.bitwise_and(img, img, mask=mask)

        if transform_to_drawings:

            blurred_image = cv2.GaussianBlur(img.copy(),(5,5),0)

            img = cv2.Canny(blurred_image, 100, 160)


        img = img[y:y2, x:x2]
        cv2.imwrite(end_path + "/" + image_name, img)
        time.sleep(0.01)
    return logs


In [14]:
WHOLE_DATA_PATH = "../data/raw_in_one_folder"
RESULT_PATH = "../data/processed_dataset"

In [15]:
for dataset in ['train', 'test']:
    folder = WHOLE_DATA_PATH + "/" + dataset
    result_folder = RESULT_PATH + "/" + dataset

    result = extract_contours(folder=folder,
                     output_dir=result_folder,
                     Segmentator=YOLOSegmentation(model_path, device),
                     transform_to_drawings=False,
                     skip_exist=True)
    print(result)


0: 480x640 2 persons, 1 car, 1331.1ms
Speed: 0.0ms preprocess, 1331.1ms inference, 15.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cars, 1332.6ms
Speed: 0.0ms preprocess, 1332.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 448x640 5 cars, 1363.5ms
Speed: 15.6ms preprocess, 1363.5ms inference, 15.6ms postprocess per image at shape (1, 3, 448, 640)

0: 480x640 1 car, 1 truck, 1433.0ms
Speed: 15.7ms preprocess, 1433.0ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 car, 1455.6ms
Speed: 0.0ms preprocess, 1455.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 1 person, 2 cars, 1 bird, 2 umbrellas, 1290.6ms
Speed: 15.6ms preprocess, 1290.6ms inference, 0.0ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 7 cars, 1354.0ms
Speed: 0.0ms preprocess, 1354.0ms inference, 15.6ms postprocess per image at shape (1, 3, 480, 640)

0: 480x640 2 cars, 1 potted plant, 1456.5ms
Speed

[]



0: 448x640 9 cars, 1290.0ms
Speed: 4.0ms preprocess, 1290.0ms inference, 17.0ms postprocess per image at shape (1, 3, 448, 640)

0: 416x640 2 persons, 1 car, 1201.0ms
Speed: 4.0ms preprocess, 1201.0ms inference, 10.0ms postprocess per image at shape (1, 3, 416, 640)

0: 448x640 1 person, 1 car, 1 truck, 1323.0ms
Speed: 4.0ms preprocess, 1323.0ms inference, 8.0ms postprocess per image at shape (1, 3, 448, 640)

0: 480x640 2 cars, 1 bus, 1394.0ms
Speed: 3.0ms preprocess, 1394.0ms inference, 9.0ms postprocess per image at shape (1, 3, 480, 640)

0: 448x640 1 car, 3 trucks, 1337.0ms
Speed: 4.0ms preprocess, 1337.0ms inference, 11.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 1 car, 1301.0ms
Speed: 4.0ms preprocess, 1301.0ms inference, 5.0ms postprocess per image at shape (1, 3, 448, 640)

0: 448x640 3 cars, 1346.0ms
Speed: 4.0ms preprocess, 1346.0ms inference, 9.0ms postprocess per image at shape (1, 3, 448, 640)

0: 384x640 1 truck, 1136.0ms
Speed: 3.0ms preprocess, 113

[]


In [ ]:
for dataset in ['val']:
    folder = WHOLE_DATA_PATH + "/" + 'test'
    result_folder = RESULT_PATH + "/" + dataset

    extract_contours(folder=folder,
                     output_dir=result_folder,
                     Segmentator=YOLOSegmentation(model_path, device),
                     transform_to_drawings=False)